# Heart Rate (Variability) Feature Merging

This notebook merges different heart rate (variability) data computed into one long-format dataframe. The different data to be merged are:
* Mean heart rate per subject, MIST phase, and subphase (in bpm)
* *Normalized* Mean heart rate per subject, MIST phase and subphase (in percent relative to the global baseline $BL_{Glo}$)
* Heart rate variability measures per subject, MIST phase, and subphase
* *Percentage* of HR(V) measures per subject, MIST phase and subphase that are above the mean value during the global baseline $BL_{Glo}$

## Imports and Helper Functions

In [ ]:
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit as bp

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

## Import Data

In [ ]:
base_path = Path("../../../data/processed")
ecg_path = base_path.joinpath("ecg")

In [ ]:
hr_data = bp.io.load_wide_format_csv(
    ecg_path.joinpath("cft_hr_mean.csv")
)

hr_data_norm = bp.io.load_wide_format_csv(
    ecg_path.joinpath("cft_hr_mean_normalized.csv")
)

hrv_data = bp.io.load_wide_format_csv(
    ecg_path.joinpath("cft_hrv_subphases.csv"), 
    index_cols=["condition", "subject", "phase", "subphase"]
)

hrv_above_baseline = bp.io.load_wide_format_csv(
    ecg_path.joinpath("cft_hrv_above_baseline.csv"), 
    index_cols=["condition", "subject", "phase", "subphase"]
)

## Merge Data

### Select Columns

In [ ]:
hrv_data = hrv_data[["HRV_SDNN", "HRV_RMSSD", "HRV_SD1SD2", "HRV_pNN50", "HRV_pNN20"]]

### Rename Columns

In [ ]:
hr_data.columns = ["HR"]
hr_data_norm.columns = ["HR_Norm"]

### Concat Dataframes, Reorder Index Levels

In [ ]:
data_merged = pd.concat([hr_data, hr_data_norm, hrv_data, hrv_above_baseline], axis=1)
#data_merged.index = data_merged.index.set_names("name", level=-1)
#data_merged = data_merged.reset_index().set_index(["condition", "subject", "phase", "subphase", "type", "name"])
data_merged.columns.name = "name"
data_merged = pd.concat({"data": data_merged.stack()}, axis=1)
data_merged

## Export Data

In [ ]:
data_merged.to_csv(ecg_path.joinpath("cft_measures_merged.csv"))